In [72]:
import pandas as pd
import numpy as np

In [73]:
data = pd.read_csv('./customer_shopping_data.csv')
data.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date
0,I138884,C241288,Female,28,Clothing,5,1500.40,Alipay,5/8/2022
1,I317333,C111565,Male,21,Shoes,3,1800.51,WeChat Pay,12/12/2021
2,I127801,C266599,Male,20,Clothing,1,300.08,Card,9/11/2021
3,I173702,C988172,Female,66,Shoes,5,3000.85,Alipay,16/05/2021
4,I337046,C189076,Female,53,Books,4,60.60,Card,24/10/2021


In [74]:
unique_values = data['payment_method'].unique().tolist()
unique_values

['Alipay', 'WeChat Pay', 'Card']

In [75]:
# 明确日期格式为：日/月/年
data['invoice_date'] = pd.to_datetime(data['invoice_date'], format = '%d/%m/%Y')

# 提取年列和月列
data['invoice_year'] = data['invoice_date'].dt.year
data['invoice_month'] = data['invoice_date'].dt.month

# 创建映射字典
gender_mapping = {
    'Female': '女',
    'Male': '男'
}

payment_mapping = {
    'Alipay': '支付宝',
    'WeChat Pay': '微信支付',
    'Card': '银行卡'
}

# 使用映射字典进行替换
data['gender'] = data['gender'].map(gender_mapping)
data['payment_method'] = data['payment_method'].map(payment_mapping)

data.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,invoice_year,invoice_month
0,I138884,C241288,女,28,Clothing,5,1500.40,支付宝,2022-08-05,2022,8
1,I317333,C111565,男,21,Shoes,3,1800.51,微信支付,2021-12-12,2021,12
2,I127801,C266599,男,20,Clothing,1,300.08,银行卡,2021-11-09,2021,11
3,I173702,C988172,女,66,Shoes,5,3000.85,支付宝,2021-05-16,2021,5
4,I337046,C189076,女,53,Books,4,60.60,银行卡,2021-10-24,2021,10


In [76]:
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   invoice_no      99457 non-null  object        
 1   customer_id     99457 non-null  object        
 2   gender          99457 non-null  object        
 3   age             99457 non-null  int64         
 4   category        99457 non-null  object        
 5   quantity        99457 non-null  int64         
 6   price           99457 non-null  float64       
 7   payment_method  99457 non-null  object        
 8   invoice_date    99457 non-null  datetime64[ns]
 9   invoice_year    99457 non-null  int32         
 10  invoice_month   99457 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), object(5)
memory usage: 7.6+ MB


(99457, 11)

In [77]:
data['revenue'] = round(data['quantity'] * data['price'], 2)
data.describe()

,age,quantity,price,invoice_date,invoice_year,invoice_month,revenue
count,99457.000000,99457.000000,99457.000000,99457,99457.000000,99457.000000,99457.000000
mean,43.427089,3.003429,689.256321,2022-02-04 02:46:59.783424,2021.629408,6.113898,2528.789268
min,18.000000,1.000000,5.230000,2021-01-01 00:00:00,2021.000000,1.000000,5.230000
25%,30.000000,2.000000,45.450000,2021-07-19 00:00:00,2021.000000,3.000000,136.350000
50%,43.000000,3.000000,203.300000,2022-02-05 00:00:00,2022.000000,6.000000,600.170000
75%,56.000000,4.000000,1200.320000,2022-08-22 00:00:00,2022.000000,9.000000,2700.720000
max,69.000000,5.000000,5250.000000,2023-03-08 00:00:00,2023.000000,12.000000,26250.000000
std,14.990054,1.413025,941.184567,NaN,0.636136,3.569511,4222.475781


In [78]:
# 检查缺失值
print('\n缺失值统计：')
print(data.isnull().sum())

# 检查重复值
print('\n重复值统计：')
duplicate_count = data.duplicated().sum()
print(f'\n删除前重复值数量：{duplicate_count}')
data = data.drop_duplicates()
print(f'\n删除后重复值数量：{data.duplicated().sum()}')


缺失值统计：
invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
invoice_year      0
invoice_month     0
revenue           0
dtype: int64

重复值统计：

删除前重复值数量：0

删除后重复值数量：0


In [ ]:
# 用户画像预处理
# 年龄画像
def age_label(age):
    if age <= 17:
        return '少年'
    elif age <= 35:
        return '青年'
    elif age <= 50:
        return '中年'
    else:
        return '老年'

# 应用年龄标签
data['age_label'] = data['age'].apply(age_label)

# 收入画像
quantile = data['revenue'].quantile([0.25, 0.5, 0.75]).tolist()

def revenue_label_quantile(revenue):
    if revenue <= quantile[0]:
        return '低消费群体'
    elif revenue <= quantile[1]:
        return '中等消费群体'
    elif revenue <= quantile[2]:
        return '中高消费群体'
    else:
        return '高消费群体'

# 应用收入标签
data['revenue_label'] = data['revenue'].apply(revenue_label_quantile)

data.head()
    

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,invoice_year,invoice_month,revenue,age_label,revenue_label
0,I138884,C241288,女,28,Clothing,5,1500.40,支付宝,2022-08-05,2022,8,7502.00,青年,高消费群体
1,I317333,C111565,男,21,Shoes,3,1800.51,微信支付,2021-12-12,2021,12,5401.53,青年,高消费群体
2,I127801,C266599,男,20,Clothing,1,300.08,银行卡,2021-11-09,2021,11,300.08,青年,中等消费群体
3,I173702,C988172,女,66,Shoes,5,3000.85,支付宝,2021-05-16,2021,5,15004.25,老年,高消费群体
4,I337046,C189076,女,53,Books,4,60.60,银行卡,2021-10-24,2021,10,242.40,老年,中等消费群体


In [80]:
data.to_csv(r'./output.csv', index = False)